In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
bat_data = pd.read_csv("Clean_Bat_2022.csv",index_col = "Unnamed: 0")
pitch_data = pd.read_csv("Clean_Pitch_2022.csv",index_col = "Unnamed: 0")
bat_data = bat_data.fillna(0)
pitch_data = pitch_data.fillna(0)
h_a = []
for ha in bat_data['Home_Away']:
    if ha == 0:
        h_a.append(ha)
    else:
        h_a.append(1)
bat_data['Home_Away'] = h_a
h_a = []
for ha in pitch_data['Home_Away']:
    if ha == 0:
        h_a.append(ha)
    else:
        h_a.append(1)
pitch_data['Home_Away'] = h_a
new_col_list = []
for col in bat_data.columns:
    if col == "Name" or col == "Date" or col == "Opp" or col == "Team" or col == "Home_Away":
        new_col_list.append(col)
    else:
        col = col + "_Bat"
        new_col_list.append(col)
bat_data.columns = new_col_list
new_col_list = []
for col in pitch_data.columns:
    if col == "Name" or col == "Date" or col == "Opp" or col == "Team" or col == "Home_Away":
        new_col_list.append(col)
    else:
        col = col + "_Pitch"
        new_col_list.append(col)
pitch_data.columns = new_col_list
## Loop through the dates, loop through the teams in the date dataframe and concatinate the data into sums
team_totals_bat = pd.DataFrame()
for date in bat_data['Date'].unique():
    game = bat_data[bat_data['Date'] == date]
    for team in game.Team.unique():
        team = game[game['Team'] == team]
        new_df = team[team.columns[5:]].sum().to_frame().T
        new_df['Team'] = team.iloc[0]['Team']
        new_df['Date'] = team.iloc[0]['Date']
        new_df['Opp'] = team.iloc[0]['Opp']
        new_df['Home_Away'] = team.iloc[0]['Home_Away']
        new_df.head()
        new_columns = ['Date','Team','Opp','Home_Away'] + list(team.columns[5:])
        new_df = new_df[new_columns]
        team_totals_bat = pd.concat([team_totals_bat,new_df])
team_totals_bat.head()

,Date,Team,Opp,Home_Away,PA_Bat,AB_Bat,R_Bat,H_Bat,2B_Bat,3B_Bat,...,RBI_Bat,BB_Bat,IBB_Bat,SO_Bat,HBP_Bat,SH_Bat,SF_Bat,GDP_Bat,SB_Bat,CS_Bat
0,"Apr 7, 2022",MIL,CHN,1,40,33,4,10,4,0,...,4,4,0,9,1,0,2,2,1,0
0,"Apr 7, 2022",LAA,HOU,0,31,29,1,4,0,1,...,1,1,0,7,1,0,0,3,0,0
0,"Apr 7, 2022",ATL,CIN,0,36,31,3,4,0,0,...,2,5,0,13,0,0,0,0,0,0
0,"Apr 7, 2022",ARI,SDN,0,34,26,4,3,0,0,...,3,7,0,6,1,0,0,1,0,0
0,"Apr 7, 2022",NYN,WSN,1,42,35,5,12,2,0,...,5,4,0,9,3,0,0,2,0,1


In [20]:
team_totals_pitch = pd.DataFrame()
for date in pitch_data.Date.unique():
    game = pitch_data[pitch_data['Date'] == date]
    for team in game.Team.unique():
        df = game[game['Team'] == team]
        bullpen = df[df['GS_Pitch'] < 1]
        start = df[df['GS_Pitch'] == 1]
        new_df = bullpen[bullpen.columns[5:]].sum().to_frame().T
        ## Need to account for double headers since there's two pitchers with GS == 1
        if str(new_df['IP_Pitch']).split("\n")[0][-1] == "3":
            new_df['IP_Pitch'] = float(new_df['IP_Pitch']) + .7
        elif str(new_df['IP_Pitch']).split("\n")[0][-1] == "6":
            new_df['IP_Pitch'] = float(new_df['IP_Pitch']) + 1.4
        elif str(new_df['IP_Pitch']).split("\n")[0][-1] == "5":
            new_df['IP_Pitch'] = float(new_df['IP_Pitch']) + 1.2
        new_df.head()
        new_df.drop(columns = ['GS_Pitch'],inplace = True)
        new_col_list = []
        for col in new_df.columns:
            col += "_Pen"
            new_col_list.append(col)
        new_df.columns = new_col_list
        for col in new_col_list:
            start[col] = new_df[col].iloc[0]
        team_totals_pitch = pd.concat([team_totals_pitch,start])
team_totals_pitch.head()

,Name,Date,Opp,Team,Home_Away,GS_Pitch,IP_Pitch,H_Pitch,R_Pitch,ER_Pitch,...,IBB_Pitch_Pen,GDP_Pitch_Pen,SF_Pitch_Pen,SB_Pitch_Pen,CS_Pitch_Pen,PO_Pitch_Pen,BF_Pitch_Pen,GB/FB_Pitch_Pen,LD_Pitch_Pen,PU_Pitch_Pen
12,Patrick Corbin,"Apr 7, 2022",NYN,WSN,0,1,4.0,5,2,2,...,0.0,2.0,0.0,0.0,0.0,0.0,24.0,2.10,1.45,0.00
9,Corbin Burnes,"Apr 7, 2022",CHN,MIL,1,1,5.0,4,3,3,...,0.0,1.0,0.0,0.0,0.0,0.0,14.0,1.00,0.17,0.17
20,Zack Greinke,"Apr 7, 2022",CLE,KCR,0,1,5.2,5,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.40,1.50,0.70
4,Shane Bieber,"Apr 7, 2022",KCR,CLE,1,1,4.2,2,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,13.0,0.44,0.11,0.11
42,Shohei Ohtani,"Apr 7, 2022",HOU,LAA,0,1,4.2,4,1,1,...,1.0,0.0,0.0,1.0,0.0,0.0,19.0,2.40,0.60,0.00


In [21]:
team_totals_pitch['Total_IP_Pitch'] = team_totals_pitch['IP_Pitch'] + team_totals_pitch['IP_Pitch_Pen']
team_totals_pitch[team_totals_pitch['Total_IP_Pitch'] > 14].head(10)

,Name,Date,Opp,Team,Home_Away,GS_Pitch,IP_Pitch,H_Pitch,R_Pitch,ER_Pitch,...,GDP_Pitch_Pen,SF_Pitch_Pen,SB_Pitch_Pen,CS_Pitch_Pen,PO_Pitch_Pen,BF_Pitch_Pen,GB/FB_Pitch_Pen,LD_Pitch_Pen,PU_Pitch_Pen,Total_IP_Pitch
77,Triston Mckenzie,"May 7, 2022",TOR,CLE,0,1,6.0,4,2,2,...,1.0,0.0,0.0,1.0,0.0,33.0,3.67,0.33,0.67,14.2
92,Connor Overton,"May 7, 2022",PIT,CIN,0,1,5.1,6,2,2,...,0.0,0.0,0.0,0.0,0.0,57.0,2.25,2.46,0.00,15.8
114,Drew Smyly,"May 7, 2022",LAN,CHN,0,1,4.1,6,3,2,...,1.0,0.0,0.0,0.0,0.0,52.0,4.31,1.23,0.08,15.5
44,Kyle Gibson,"May 8, 2022",NYN,PHI,0,1,6.0,6,2,2,...,0.0,0.0,0.0,0.0,0.0,42.0,2.42,1.10,2.18,15.2
73,Germán Márquez,"May 21, 2022",NYN,COL,0,1,6.0,11,5,4,...,0.0,0.0,0.0,1.0,0.0,33.0,2.21,1.71,0.00,15.0
4,Joan Adon,"May 28, 2022",COL,WSN,0,1,6.0,6,2,0,...,0.0,0.0,0.0,0.0,0.0,35.0,2.21,1.85,0.29,14.1
72,Chad Kuhl,"May 28, 2022",WSN,COL,1,1,6.1,7,2,2,...,1.0,0.0,2.0,0.0,0.0,40.0,1.56,2.46,0.47,14.5
44,Nathan Eovaldi,"May 28, 2022",BAL,BOS,0,1,9.0,7,3,2,...,1.0,0.0,0.0,0.0,0.0,20.0,2.67,0.00,2.33,15.0
2,Aaron Ashby,"May 30, 2022",CHN,MIL,1,1,6.0,5,1,1,...,2.0,0.0,3.0,0.0,0.0,37.0,3.04,1.52,0.75,15.1
90,Matt Swarmer,"May 30, 2022",MIL,CHN,0,1,6.0,5,4,1,...,1.0,0.0,0.0,0.0,0.0,39.0,2.04,0.92,0.00,15.0


In [15]:
team_totals_bat[team_totals_bat['AB_Bat'] > 60].head(20)

,Date,Team,Opp,Home_Away,PA_Bat,AB_Bat,R_Bat,H_Bat,2B_Bat,3B_Bat,...,RBI_Bat,BB_Bat,IBB_Bat,SO_Bat,HBP_Bat,SH_Bat,SF_Bat,GDP_Bat,SB_Bat,CS_Bat
0,"Apr 19, 2022",ARI,WSN,1,69,63,1,11,3,0,...,1,6,0,21,0,0,0,1,0,1
0,"Apr 19, 2022",NYN,SFN,0,73,66,8,16,5,0,...,7,6,0,16,0,1,0,1,2,0
0,"Apr 19, 2022",SFN,NYN,1,74,67,5,11,1,0,...,5,7,0,18,0,0,0,1,1,1
0,"Apr 20, 2022",CLE,CHA,0,73,67,13,17,3,0,...,13,6,0,16,0,0,0,1,0,0
0,"Apr 23, 2022",DET,COL,0,81,75,15,26,1,1,...,15,6,0,18,0,0,0,1,1,0
0,"Apr 23, 2022",COL,DET,1,67,62,3,10,0,0,...,3,5,0,21,0,0,0,2,0,0
0,"May 3, 2022",ATL,NYN,1,71,64,4,12,4,0,...,4,6,0,17,0,0,1,2,0,0
0,"May 3, 2022",NYN,ATL,0,67,61,8,17,1,0,...,8,4,0,9,1,0,1,2,1,1
0,"May 4, 2022",SDN,CLE,1,82,66,10,11,4,0,...,10,15,2,23,0,0,1,1,1,0
0,"May 4, 2022",DET,PIT,0,70,65,5,11,5,0,...,3,4,1,19,0,0,1,0,1,0


In [16]:
team_totals_bat.columns

Index(['Date', 'Team', 'Opp', 'Home_Away', 'PA_Bat', 'AB_Bat', 'R_Bat',
       'H_Bat', '2B_Bat', '3B_Bat', 'HR_Bat', 'RBI_Bat', 'BB_Bat', 'IBB_Bat',
       'SO_Bat', 'HBP_Bat', 'SH_Bat', 'SF_Bat', 'GDP_Bat', 'SB_Bat', 'CS_Bat'],
      dtype='object')

In [6]:
team_totals_bat.to_csv("Team_Bat_Stats_2022.csv")
team_totals_pitch.to_csv("Team_Pitch_Stats_2022.csv")

In [ ]:
avg = round(new_df.H_Bat/new_df.AB_Bat,3)
avg

In [ ]:
## Start with the team
## Take a date past seven days into the season
## Take the dataframe where the date is less than that date
## Take the dataframe that is greater than seven days prior
## Got to make the date column into a datetime foramt to make it easier to compare the dates, strings aren't working well

,Name,Date,Opp,Team,Home_Away,GS_Pitch,IP_Pitch,H_Pitch,R_Pitch,ER_Pitch,...,IBB_Pitch_Pen,GDP_Pitch_Pen,SF_Pitch_Pen,SB_Pitch_Pen,CS_Pitch_Pen,PO_Pitch_Pen,BF_Pitch_Pen,GB/FB_Pitch_Pen,LD_Pitch_Pen,PU_Pitch_Pen
50,Luis Garcia,"Apr 12, 2022",ARI,HOU,1,1,4.0,2,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,20.0,2.42,0.83,0.25
